In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
pd.set_option('display.max_columns', 50)

In [38]:
row_count = 50000
df = pd.DataFrame()
for chunk in pd.read_csv('IndividualsAndHouseholdsProgramValidRegistrationsV2.csv',
                 chunksize=row_count,
                 dtype=str):
    temp = chunk[[
    'incidentTypeCode',
    'disasterNumber',
    'county',
    'fips',
    'damagedStateAbbreviation',
    'ownRent',
    'rpfvl',
    'renterDamageLevel',
    ]]
    temp = temp[temp['incidentTypeCode']=='H']
    df = pd.concat([df, temp],
        ignore_index=True,
        copy=False)
    print(temp.shape, df.shape)

(0, 8) (0, 8)
(0, 8) (0, 8)
(0, 8) (0, 8)
(15662, 8) (15662, 8)
(0, 8) (15662, 8)
(13235, 8) (28897, 8)
(50000, 8) (78897, 8)
(50000, 8) (128897, 8)
(50000, 8) (178897, 8)
(4424, 8) (183321, 8)
(0, 8) (183321, 8)
(0, 8) (183321, 8)
(0, 8) (183321, 8)
(0, 8) (183321, 8)
(0, 8) (183321, 8)
(30484, 8) (213805, 8)
(50000, 8) (263805, 8)
(50000, 8) (313805, 8)
(50000, 8) (363805, 8)
(50000, 8) (413805, 8)
(39285, 8) (453090, 8)
(50000, 8) (503090, 8)
(50000, 8) (553090, 8)
(50000, 8) (603090, 8)
(50000, 8) (653090, 8)
(50000, 8) (703090, 8)
(50000, 8) (753090, 8)
(50000, 8) (803090, 8)
(50000, 8) (853090, 8)
(50000, 8) (903090, 8)
(50000, 8) (953090, 8)
(50000, 8) (1003090, 8)
(50000, 8) (1053090, 8)
(50000, 8) (1103090, 8)
(50000, 8) (1153090, 8)
(50000, 8) (1203090, 8)
(50000, 8) (1253090, 8)
(761, 8) (1253851, 8)
(0, 8) (1253851, 8)
(0, 8) (1253851, 8)
(2822, 8) (1256673, 8)
(39574, 8) (1296247, 8)
(42760, 8) (1339007, 8)
(50000, 8) (1389007, 8)
(50000, 8) (1439007, 8)
(50000, 8) (148900

In [39]:
df.head()

,incidentTypeCode,disasterNumber,county,fips,damagedStateAbbreviation,ownRent,rpfvl,renterDamageLevel
0,H,1479,Victoria (County),NaN,TX,O,412.05,NaN
1,H,1479,Bee (County),NaN,TX,O,940.84,NaN
2,H,1479,Matagorda (County),NaN,TX,O,22724.04,NaN
3,H,1479,Calhoun (County),NaN,TX,O,0.00,NaN
4,H,1479,Victoria (County),NaN,TX,O,928.20,NaN


In [40]:
df.to_parquet('agg_renter_level.parquet')

In [41]:
df.renterDamageLevel.describe()

count     464984
unique         4
top         5530
freq      387771
Name: renterDamageLevel, dtype: object

In [42]:
df.renterDamageLevel.value_counts()

renterDamageLevel
5530    387771
5532     52319
5531     20223
5533      4671
Name: count, dtype: int64

In [43]:
df.shape

(19486671, 8)

In [44]:
df['fips'].isna().sum()

np.int64(7849379)

In [45]:
df['fips'].value_counts()

fips
48201    1087066
12086     596552
12057     528287
12103     451672
12071     347916
          ...   
13135          1
13133          1
13023          1
13197          1
37035          1
Name: count, Length: 629, dtype: int64

In [46]:
df.incidentTypeCode.value_counts()

incidentTypeCode
H    19486671
Name: count, dtype: int64

In [47]:
print(df['incidentTypeCode'].isna().sum())
df['disasterNumber'].isna().sum()


0


np.int64(0)

In [48]:
len(df['disasterNumber'].unique())

99

In [49]:
df['county'] = df['county'].str.replace('(County)', '')
df['county'] = df['county'].str.replace('(Parish)', '')
df['county'] = df['county'].str.replace('.', '')
df['county'] = df['county'].str.replace('desoto', 'de soto')
df['county'] = df['county'].str.replace('dekalb', 'de kalb')
df['county'] = df['county'].str.replace('\s*\(in.*?\)', '', regex=True)
df['county'] = df['county'].str.replace('msa.*', '', regex=True)
# df['county'] = re.sub(r'\s*\(in.*?)', '', df['county'])
df['county'] = df['county'].str.strip()
df['county'] = df['county'].str.lower()

<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
/var/folders/15/mh2mgzhn1191h4zq_5d03wfc0000gn/T/ipykernel_49814/1036810952.py:6: SyntaxWarning: invalid escape sequence '\s'
  df['county'] = df['county'].str.replace('\s*\(in.*?\)', '', regex=True)


In [50]:
df['county'].value_counts()

county
harris                              1611886
jefferson                            927940
miami-dade                           800297
orleans                              643820
hillsborough                         578725
                                     ...   
harley                                    1
union mills                               1
alpine beach                              1
alliance (rr name west alliance)          1
ontario                                   1
Name: count, Length: 731, dtype: int64

In [51]:
df.dtypes

incidentTypeCode            object
disasterNumber              object
county                      object
fips                        object
damagedStateAbbreviation    object
ownRent                     object
rpfvl                       object
renterDamageLevel           object
dtype: object

In [52]:
fips_df = pd.read_csv('State__County_and_City_FIPS_Reference_Table_20250531.csv',
                    dtype=str)
fips_df.head()

,State Name,County Name,City Name,State Code,State FIPS Code,County Code,StCnty FIPS Code,City Code,StCntyCity FIPS Code
0,ALABAMA,AUTAUGA,AUTAUGAVILLE,AL,01,C001,01001,0220,010010220
1,ALABAMA,AUTAUGA,BILLINGSLEY,AL,01,C001,01001,0340,010010340
2,ALABAMA,AUTAUGA,BOOTH,AL,01,C001,01001,0425,010010425
3,ALABAMA,AUTAUGA,JONES,AL,01,C001,01001,1796,010011796
4,ALABAMA,AUTAUGA,MARBURY,AL,01,C001,01001,2002,010012002


In [53]:
fips_df = fips_df[[
    'State Code',
    'County Name',
    'StCnty FIPS Code',
    ]]
fips_df = fips_df.rename(columns={
    'State Code':'damagedStateAbbreviation',
    'County Name':'county',
    'StCnty FIPS Code':'fipsCode'
})
fips_df['county'] = fips_df['county'].str.lower()
fips_df = fips_df.drop_duplicates()
fips_df.head()

,damagedStateAbbreviation,county,fipsCode
0,AL,autauga,01001
6,AL,baldwin,01003
30,AL,barbour,01005
38,AL,bibb,01007
47,AL,blount,01009


In [54]:
combined_df = pd.merge(df,
                       fips_df,
                       on=['damagedStateAbbreviation', 'county'],
                       how='left')
print(df.shape, combined_df.shape)

(19486671, 8) (19488406, 9)


In [55]:
combined_df['fipsCode'] = np.where((combined_df['fipsCode'].isna())&(combined_df['fips'].notna()),
                                   combined_df['fips'],
                                   combined_df['fipsCode'])
combined_df['fipsCode'] = np.where((combined_df['fipsCode'].isna())&(
    combined_df['damagedStateAbbreviation']=='DC'),
                                   '11001',
                                   combined_df['fipsCode'])

In [56]:
combined_df['fipsCode'].isna().sum()

np.int64(1321730)

In [57]:
combined_df[(combined_df['fipsCode'].isna())&(
    combined_df['damagedStateAbbreviation']=='GA'
)]


,incidentTypeCode,disasterNumber,county,fips,damagedStateAbbreviation,ownRent,rpfvl,renterDamageLevel,fipsCode
1263285,H,1554,dekalb,NaN,GA,O,1053.06,NaN,NaN
1263294,H,1554,dekalb,NaN,GA,O,5100.00,NaN,NaN
1263295,H,1554,dekalb,NaN,GA,O,0.00,NaN,NaN
1263299,H,1554,dekalb,NaN,GA,O,0.00,NaN,NaN
1263300,H,1554,dekalb,NaN,GA,O,0.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1265928,H,1554,dekalb,NaN,GA,R,0.00,NaN,NaN
1265931,H,1554,dekalb,NaN,GA,O,1275.00,NaN,NaN
1265936,H,1554,dekalb,NaN,GA,R,0.00,NaN,NaN
1265937,H,1554,dekalb,NaN,GA,O,567.87,NaN,NaN


In [58]:
combined_df[combined_df['fipsCode'].isna()].sample(20)

,incidentTypeCode,disasterNumber,county,fips,damagedStateAbbreviation,ownRent,rpfvl,renterDamageLevel,fipsCode
14818472,H,4671,ponce (municipio),NaN,PR,O,3115.91,NaN,NaN
14044611,H,4671,barceloneta (municipio),NaN,PR,O,0.00,NaN,NaN
14921268,H,4671,vega baja (municipio),NaN,PR,O,0.00,NaN,NaN
14289611,H,4671,arecibo (municipio),NaN,PR,R,0.00,NaN,NaN
15010912,H,4671,vega alta (municipio),NaN,PR,R,0.00,NaN,NaN
14927545,H,4671,san juan (municipio),NaN,PR,R,0.00,NaN,NaN
15028321,H,4671,bayamon (municipio),NaN,PR,R,0.00,NaN,NaN
14327175,H,4671,loiza (municipio),NaN,PR,O,0.00,NaN,NaN
13985980,H,4671,toa alta (municipio),NaN,PR,O,0.00,NaN,NaN
1263874,H,1554,dekalb,NaN,GA,O,2550.00,NaN,NaN


In [59]:
combined_df.head()

,incidentTypeCode,disasterNumber,county,fips,damagedStateAbbreviation,ownRent,rpfvl,renterDamageLevel,fipsCode
0,H,1479,victoria,NaN,TX,O,412.05,NaN,48469
1,H,1479,bee,NaN,TX,O,940.84,NaN,48025
2,H,1479,matagorda,NaN,TX,O,22724.04,NaN,48321
3,H,1479,calhoun,NaN,TX,O,0.00,NaN,48057
4,H,1479,victoria,NaN,TX,O,928.20,NaN,48469


In [60]:
combined_df['disasterNumber'].value_counts()

disasterNumber
4337    2627039
4671    1239103
4339    1120768
4798    1082971
1603    1076989
         ...   
1494        667
4089        592
4026        513
4844         30
1495          1
Name: count, Length: 99, dtype: int64

In [61]:
combined_df['rpfvl'] = combined_df['rpfvl'].astype(float)

In [62]:
combined_df[combined_df['rpfvl']>=20000].head(20)


,incidentTypeCode,disasterNumber,county,fips,damagedStateAbbreviation,ownRent,rpfvl,renterDamageLevel,fipsCode
2,H,1479,matagorda,NaN,TX,O,22724.04,NaN,48321
163,H,1479,galveston,NaN,TX,O,38771.78,NaN,48167
2132,H,1479,galveston,NaN,TX,O,38418.83,NaN,48167
2395,H,1479,galveston,NaN,TX,O,39688.83,NaN,48167
5551,H,1479,brazoria,NaN,TX,O,20757.67,NaN,48039
15795,H,1490,carteret,NaN,NC,O,24338.78,NaN,37031
15820,H,1490,carteret,NaN,NC,O,21252.72,NaN,37031
15830,H,1490,beaufort,NaN,NC,O,26375.25,NaN,37013
15887,H,1490,chowan,NaN,NC,O,20446.60,NaN,37041
15932,H,1490,chowan,NaN,NC,O,29242.83,NaN,37041


In [63]:
combined_df['majorDamageGeneral'] = np.where(combined_df['rpfvl']>=20000,
                                             1,
                                             0)
combined_df['majorDamageOwn'] = np.where((combined_df['rpfvl']>=20000)&(combined_df['ownRent']=='O'),
                                             1,
                                             0)
combined_df['majorDamageRent'] = np.where((combined_df['rpfvl']>=20000)&(combined_df['ownRent']=='R'),
                                             1,
                                             0)
combined_df['validReg_own'] = np.where((combined_df['ownRent']=='O'),
                                             1,
                                             0)
combined_df['validReg_rent'] = np.where((combined_df['ownRent']=='R'),
                                             1,
                                             0)
combined_df.sample(20)

,incidentTypeCode,disasterNumber,county,fips,damagedStateAbbreviation,ownRent,rpfvl,renterDamageLevel,fipsCode,majorDamageGeneral,majorDamageOwn,majorDamageRent,validReg_own,validReg_rent
17219054,H,4798,harris,48201,TX,R,0.00,NaN,48201,0,0,0,0,1
14424294,H,4671,quebradillas (municipio),NaN,PR,O,0.00,NaN,NaN,0,0,0,1,0
7629100,H,4332,tyler,48457,TX,O,0.00,NaN,48457,0,0,0,1,0
8489976,H,4337,hillsborough,12057,FL,O,0.00,NaN,12057,0,0,0,1,0
14868958,H,4671,ponce (municipio),NaN,PR,O,0.00,NaN,NaN,0,0,0,1,0
16272653,H,4798,montgomery,NaN,TX,O,0.00,NaN,48339,0,0,0,1,0
15491180,H,4673,pinellas,12103,FL,R,0.00,NaN,12103,0,0,0,0,1
2282714,H,1603,st bernard,NaN,LA,R,0.00,NaN,22087,0,0,0,0,1
1074751,H,1549,mobile,NaN,AL,O,887.08,NaN,01097,0,0,0,1,0
6266619,H,4021,essex,34013,NJ,O,834.92,NaN,34013,0,0,0,1,0


In [64]:
combined_df = pd.get_dummies(combined_df,
               columns=['renterDamageLevel'],
               dtype=int)

In [65]:
combined_df.head()

,incidentTypeCode,disasterNumber,county,fips,damagedStateAbbreviation,ownRent,rpfvl,fipsCode,majorDamageGeneral,majorDamageOwn,majorDamageRent,validReg_own,validReg_rent,renterDamageLevel_5530,renterDamageLevel_5531,renterDamageLevel_5532,renterDamageLevel_5533
0,H,1479,victoria,NaN,TX,O,412.05,48469,0,0,0,1,0,0,0,0,0
1,H,1479,bee,NaN,TX,O,940.84,48025,0,0,0,1,0,0,0,0,0
2,H,1479,matagorda,NaN,TX,O,22724.04,48321,1,1,0,1,0,0,0,0,0
3,H,1479,calhoun,NaN,TX,O,0.00,48057,0,0,0,1,0,0,0,0,0
4,H,1479,victoria,NaN,TX,O,928.20,48469,0,0,0,1,0,0,0,0,0


In [66]:
combined_df.to_parquet('agg_combined_fips_renter_level.parquet')


In [67]:
combined_df_agg = combined_df.groupby([
        'incidentTypeCode',
        'disasterNumber',
        'damagedStateAbbreviation',
        'county',
        'fipsCode',
    ],
        dropna=False).agg(
            total_count = ('majorDamageGeneral', 'count'),
            validReg_own = ('validReg_own', 'sum'),
            validReg_rent = ('validReg_rent', 'sum'),
            majorDamageGeneral_total = ('majorDamageGeneral', 'sum'),
            majorDamageOwn_total = ('majorDamageOwn', 'sum'),
            majorDamageRent_total = ('majorDamageRent', 'sum'),
            renterDamageLevel_5530_total = ('renterDamageLevel_5530', 'sum'),
            renterDamageLevel_5531_total = ('renterDamageLevel_5531', 'sum'),
            renterDamageLevel_5532_total = ('renterDamageLevel_5532', 'sum'),
            renterDamageLevel_5533_total = ('renterDamageLevel_5533', 'sum'),
        ).reset_index()
combined_df_agg.sample(30)

,incidentTypeCode,disasterNumber,damagedStateAbbreviation,county,fipsCode,total_count,validReg_own,validReg_rent,majorDamageGeneral_total,majorDamageOwn_total,majorDamageRent_total,renterDamageLevel_5530_total,renterDamageLevel_5531_total,renterDamageLevel_5532_total,renterDamageLevel_5533_total
224,H,1539,FL,volusia,12127,28541,22300,4491,38,37,1,0,0,0,0
453,H,1553,NC,rockingham,37157,19,19,0,1,1,0,0,0,0,0
373,H,1549,AL,talladega,01121,96,83,9,1,1,0,0,0,0,0
1854,H,4394,SC,marlboro,45069,1711,1366,335,2,2,0,52,0,2,0
1444,H,4332,LA,evangeline,22039,1,0,1,0,0,0,0,0,0,0
2161,H,4671,PR,juana diaz (municipio),72057,1,0,1,0,0,0,0,0,0,0
1872,H,4400,GA,decatur,13087,4417,2927,1465,12,12,0,172,0,40,5
2171,H,4671,PR,lajas (municipio),72097,1,0,1,0,0,0,0,0,0,0
895,H,1609,FL,collier,12021,15760,12578,2742,53,34,19,0,0,0,0
1223,H,4025,PA,wyoming,42131,234,205,28,11,11,0,0,0,0,0


In [ ]:
combined_df_agg.shape

(2988, 15)

In [69]:
combined_df_agg.to_parquet('combined_df_agg_v2.parquet')

In [70]:
len(combined_df_agg['disasterNumber'].unique())

99

In [71]:
combined_df_agg[combined_df_agg['disasterNumber']=='1791']

,incidentTypeCode,disasterNumber,damagedStateAbbreviation,county,fipsCode,total_count,validReg_own,validReg_rent,majorDamageGeneral_total,majorDamageOwn_total,majorDamageRent_total,renterDamageLevel_5530_total,renterDamageLevel_5531_total,renterDamageLevel_5532_total,renterDamageLevel_5533_total
967,H,1791,TX,angelina,48005,4456,3231,1172,10,6,4,0,0,0,0
968,H,1791,TX,austin,48015,111,63,47,1,1,0,0,0,0,0
969,H,1791,TX,brazoria,48039,37956,26279,11357,187,143,44,0,0,0,0
970,H,1791,TX,chambers,48071,8009,6214,1703,508,353,155,0,0,0,0
971,H,1791,TX,cherokee,48073,1146,892,238,11,7,4,0,0,0,0
972,H,1791,TX,fort bend,48157,27502,20485,6858,31,26,5,0,0,0,0
973,H,1791,TX,galveston,48167,72214,39383,31810,6593,4727,1865,0,0,0,0
974,H,1791,TX,gregg,48183,270,205,57,0,0,0,0,0,0,0
975,H,1791,TX,grimes,48185,1401,918,475,8,8,0,0,0,0,0
976,H,1791,TX,hardin,48199,11746,8548,3136,71,57,14,0,0,0,0


In [72]:
combined_df_agg['damagedStateAbbreviation'].value_counts()

damagedStateAbbreviation
FL    821
PR    539
LA    272
NC    236
TX    198
SC    141
GA    134
MS    133
VA    107
AL    106
PA     77
NJ     55
NY     55
CT     34
MD     26
VT     13
VI     10
AS      5
DE      4
RI      3
MA      3
GU      2
MO      2
WV      2
NH      2
OK      1
DC      1
TN      1
AR      1
KY      1
IL      1
MI      1
OH      1
Name: count, dtype: int64

In [73]:
top_10 = ['TX', 'FL', 'SC', 'PR', 'MS', 'NC', 'LA', 'AL', 'GA', 'VA']
combined_df_agg[combined_df_agg['damagedStateAbbreviation'].isin(top_10)]['disasterNumber'].value_counts()

disasterNumber
4675    279
4671    261
4339    172
4834    112
1491    101
       ... 
1593      4
4563      4
1780      3
4677      3
1493      1
Name: count, Length: 75, dtype: int64

In [74]:
combined_df_agg[combined_df_agg['damagedStateAbbreviation'].isin(top_10)].shape

(2687, 15)

In [80]:
combined_df_agg[(combined_df_agg['damagedStateAbbreviation'].isin(top_10))&(
    combined_df_agg['disasterNumber']=='4671'
)]

,incidentTypeCode,disasterNumber,damagedStateAbbreviation,county,fipsCode,total_count,validReg_own,validReg_rent,majorDamageGeneral_total,majorDamageOwn_total,majorDamageRent_total,renterDamageLevel_5530_total,renterDamageLevel_5531_total,renterDamageLevel_5532_total,renterDamageLevel_5533_total
2044,H,4671,PR,adjuntas (municipio),72001,461,280,181,0,0,0,0,0,0,0
2045,H,4671,PR,adjuntas (municipio),72113,1,1,0,0,0,0,0,0,0,0
2046,H,4671,PR,adjuntas (municipio),NaN,6466,4291,2169,1,1,0,17,0,0,0
2047,H,4671,PR,aguada (municipio),72003,531,349,182,0,0,0,0,0,0,0
2048,H,4671,PR,aguada (municipio),NaN,14280,10445,3831,0,0,0,35,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2300,H,4671,PR,yabucoa (municipio),NaN,12383,9258,3115,0,0,0,59,0,2,0
2301,H,4671,PR,yauco (municipio),72059,1,0,1,0,0,0,0,0,0,0
2302,H,4671,PR,yauco (municipio),72127,1,1,0,0,0,0,0,0,0,0
2303,H,4671,PR,yauco (municipio),72153,618,426,191,0,0,0,3,0,0,0
